# ETL: Silver to Gold

Como já explicado anteriormente, o ETL é o processo de extração, transformação e carga de dados. Nesse notebook, vamos focar na transformação e carga dos dados do nível Silver para o nível Gold do nosso Data Warehouse. Dessa forma , estaremos preparando os dados para análises, possibilitando a utilização de ferramentas de BI para gerar os relatórios.

## 1. Configuração inicial

Essa célula importa todas as bibliotecas necessárias, define os caminhos para os arquivos de configuração (`.env`) e DDL (`gold_ddl.sql`), e cria a função `get_connection()` para se conectar ao PostgreSQL.

In [ ]:
import pandas as pd
import numpy as np
import psycopg2
from psycopg2 import sql
from psycopg2.extras import execute_values
from dotenv import load_dotenv
import os
from pathlib import Path

dotenv_path = Path('../../.env')
ddl_file_path = Path('../../data_layer/gold/gold_ddl.sql')
load_dotenv(dotenv_path=dotenv_path)

def get_connection():
    try:
        return psycopg2.connect(
            host=os.getenv('DB_HOST','localhost'),
            database=os.getenv('POSTGRES_DB','postgres'),
            user=os.getenv('POSTGRES_USER','postgres'),
            password=os.getenv('POSTGRES_PASSWORD','postgres'),
            port=os.getenv('DB_PORT', 5432)
        )
    except psycopg2.Error as e:
        print(f"Falha ao conectar ao banco de dados: {e}")
        raise

print("Configuração inicial concluída.")

## 2. Execução do DDL

Essa célula garante que nosso *Data Warehouse* (o *schema* `dw`) exista e esteja no estado correto. Efetuamos a leitura do arquivo `gold_ddl.sql` e o executamos. Dentro do DDL, usamos comando como `DROP TABLE IF EXISTS ...` e `CREATE TABLE IF NOT EXISTS` para garantir que o notebook possa ser executado várias vezes sem erros.

Algumas observações importantes sobre o DDL:

1. **Modelo Star Schema:** A arquitetura é composta por uma tabela fato_metricas_crpt (contendo as métricas numéricas, como valor de capitalização de mercado) e três dimensões de contexto (dim_crpt, dim_data, dim_hora).

2. **Chave Natural:** Ao analisar a camada silver, é possível observar que a coluna symbol estava corrompida (ex: 'USD' para todas as moedas), o que não condiz com a descrição das colunas no dataset. Dessa forma, optamos por utilizar a coluna name como chave natural. Assim, para um determinado snapshot (data e hora), cada criptomoeda é unicamente identificada pelo seu nome.


In [ ]:
print(f"Executando DDL...")
try:
    with open(ddl_file_path, 'r') as f:
        ddl_script = f.read()
    
    with get_connection() as conn, conn.cursor() as cur:
        cur.execute(ddl_script)
        conn.commit()
    
    print("Schema 'dw' e tabelas criadas com sucesso.")
except Exception as e:
    print(f"Erro ao executar DDL: {e}")
    raise

## 3. Extract

Na etapa de extração, lemos todos os dados do nosso Data Lakehouse da Camada Silver (`public.currencies_data`) e passamos para um DataFrame do Pandas (`df_silver`). Dessa forma, realizamos a extração completa dos dados, preparando-os para as etapas subsequentes com os dados em memória.

In [ ]:
print("Extraindo dados da tabela 'public.currencies_data'...")
df_silver = None
try:
    with get_connection() as conn:
        df_silver = pd.read_sql_query("SELECT * FROM public.currencies_data", conn)
    
    if df_silver is None or df_silver.empty:
        raise Exception("A tabela Silver 'public.currencies_data' está vazia ou não foi carregada.")
    
    print(f"{len(df_silver)} linhas extraídas da Silver.")

except Exception as e:
    print(f"Erro ao extrair dados da Silver: {e}")
    raise

## 4. Transform

Para iniciar a etapa de transformação, criamos três DataFrames do Panda que vão compor as nossas dimensões no Data Warehouse:

1.  **`df_dim_data`**: Extrai as datas únicas de `last_updated` e cria atributos (ano, mês, trimestre).
2.  **`df_dim_hora`**: Gera programaticamente os 86.400 segundos de um dia.
3.  **`df_dim_crpt`**: Extrai as moedas únicas (baseado no `name`), trata os valores `Infinity` (convertendo para `NaN`) e renomeia as colunas.

Depois, criamos o DataFrame da Fato (`df_fato`), que contém as métricas numéricas associadas a cada criptomoeda em um determinado snapshot (data e hora). Para isso, fazemos *merge* com as dimensões para obter os *surrogate keys* (SKs) correspondentes.

### 4.1 Transform das Dimensões

In [ ]:
print("Transformando dw.dim_data...")
df_dim_data = pd.DataFrame(pd.to_datetime(df_silver['last_updated']).dt.date.unique(), columns=['dt_cpta'])
df_dim_data = df_dim_data.dropna()
df_dim_data['sk_data'] = df_dim_data['dt_cpta'].apply(lambda x: int(x.strftime('%Y%m%d')))
df_dim_data['nr_dia'] = df_dim_data['dt_cpta'].apply(lambda x: x.day)
df_dim_data['nr_mes'] = df_dim_data['dt_cpta'].apply(lambda x: x.month)
df_dim_data['nr_ano'] = df_dim_data['dt_cpta'].apply(lambda x: x.year)
df_dim_data['nm_mes'] = df_dim_data['dt_cpta'].apply(lambda x: x.strftime('%B'))
df_dim_data['nm_d_sem'] = df_dim_data['dt_cpta'].apply(lambda x: x.strftime('%A'))
df_dim_data['nr_trim'] = (df_dim_data['nr_mes'] - 1) // 3 + 1
df_dim_data['nr_sem'] = (df_dim_data['nr_mes'] - 1) // 6 + 1
df_dim_data['fl_fds'] = df_dim_data['dt_cpta'].apply(lambda x: x.weekday() >= 5)

df_dim_data = df_dim_data[['sk_data', 'dt_cpta', 'nr_dia', 'nr_mes', 'nr_ano', 'nm_mes', 'nm_d_sem', 'nr_trim', 'nr_sem', 'fl_fds']]

print("Transformando dw.dim_hora...")
def get_periodo_dia(h):
    return ('Manhã' if 6 <= h < 12 else 'Tarde' if 12 <= h < 18 else 'Noite' if 18 <= h < 24 else 'Madrugada')
times = pd.date_range('00:00:00', '23:59:59', freq='S').time
df_dim_hora = pd.DataFrame(times, columns=['hr_cpta'])
df_dim_hora['sk_hora'] = df_dim_hora['hr_cpta'].apply(lambda x: x.hour * 10000 + x.minute * 100 + x.second)
df_dim_hora['nr_hora'] = [t.hour for t in times]
df_dim_hora['nr_min'] = [t.minute for t in times]
df_dim_hora['nr_seg'] = [t.second for t in times]
df_dim_hora['nm_perdd'] = df_dim_hora['nr_hora'].apply(get_periodo_dia)
df_dim_hora = df_dim_hora[['sk_hora', 'hr_cpta', 'nr_hora', 'nr_min', 'nr_seg', 'nm_perdd']]

print("Transformando dw.dim_crpt...")
cols = ['name', 'symbol', 'max_supply', 'is_active', 'date_added']
df_dim_crpt = df_silver[cols].drop_duplicates(subset=['name']).dropna(subset=['name']).copy()
df_dim_crpt.rename(columns={
    'name': 'nk_nome', 'symbol': 'cd_symbol', 'max_supply': 'vlr_max_supply',
    'is_active': 'fl_ativa', 'date_added': 'dt_add'
}, inplace=True)
df_dim_crpt['vlr_max_supply'] = df_dim_crpt['vlr_max_supply'].replace([np.inf, -np.inf], np.nan)
df_dim_crpt['dt_add'] = pd.to_datetime(df_dim_crpt['dt_add']).dt.date
df_dim_crpt = df_dim_crpt[['nk_nome', 'cd_symbol', 'vlr_max_supply', 'fl_ativa', 'dt_add']]

print(f"{len(df_dim_data)} datas, {len(df_dim_hora)} horas, {len(df_dim_crpt)} criptos únicas.")

## 5. Load das Dimensões

Para realizar a load das dimensões, carregamos os 3 DataFrames nas suas tabelas `dw.*`. Além disso, adicionamos um registro 'Desconhecido' (com SK = -1 ou 'N/A') em cada dimensão para garantir a integridade referencial, caso a Fato tenha uma chave que não foi encontrada.

Armazenamos essas chaves 'desconhecidas' na variável `unknowns`.

In [ ]:
print("Carregando dimensões no schema dw...")
unknowns = {}
try:
    with get_connection() as conn, conn.cursor() as cur:
        execute_values(cur, sql.SQL("""
            INSERT INTO dw.dim_data (sk_data, dt_cpta, nr_dia, nr_mes, nr_ano, nm_mes, nm_d_sem, nr_trim, nr_sem, fl_fds)
            VALUES %s ON CONFLICT (sk_data) DO NOTHING
        """), df_dim_data.values.tolist())
        cur.execute("INSERT INTO dw.dim_data (sk_data, dt_cpta) VALUES (-1, '1900-01-01') ON CONFLICT (sk_data) DO NOTHING RETURNING sk_data")
        unknowns['data'] = (cur.fetchone() or [-1])[0]

        cur.execute("SELECT COUNT(*) FROM dw.dim_hora")
        if cur.fetchone()[0] < 86400:
            execute_values(cur, sql.SQL("""
                INSERT INTO dw.dim_hora (sk_hora, hr_cpta, nr_hora, nr_min, nr_seg, nm_perdd)
                VALUES %s ON CONFLICT (sk_hora) DO NOTHING
            """), df_dim_hora.values.tolist(), page_size=5000)
        cur.execute("INSERT INTO dw.dim_hora (sk_hora, hr_cpta) VALUES (-1, '00:00:00') ON CONFLICT (sk_hora) DO NOTHING RETURNING sk_hora")
        unknowns['hora'] = (cur.fetchone() or [-1])[0]

        print("Carregando dw.dim_crpt...")
        execute_values(cur, sql.SQL("""
            INSERT INTO dw.dim_crpt (nk_nome, cd_symbol, vlr_max_supply, fl_ativa, dt_add)
            VALUES %s ON CONFLICT (nk_nome) DO UPDATE SET
                cd_symbol = EXCLUDED.cd_symbol,
                vlr_max_supply = EXCLUDED.vlr_max_supply,
                fl_ativa = EXCLUDED.fl_ativa,
                dt_add = EXCLUDED.dt_add
        """), df_dim_crpt.where(pd.notna(df_dim_crpt), None).values.tolist())
        
        cur.execute("INSERT INTO dw.dim_crpt (nk_nome) VALUES ('N/A') ON CONFLICT (nk_nome) DO NOTHING RETURNING sk_crpt")
        unknown_crpt_key_result = cur.fetchone()
        if unknown_crpt_key_result:
            unknowns['crpt'] = unknown_crpt_key_result[0]
        else:
            cur.execute("SELECT sk_crpt FROM dw.dim_crpt WHERE nk_nome = 'N/A'")
            unknowns['crpt'] = cur.fetchone()[0]
        
        conn.commit()
    
    print(f"Dimensões carregadas. Unknowns: {unknowns}")

except Exception as e:
    print(f"Erro ao carregar dimensões: {e}")
    raise

## 6. Transform e Load da Tabela Fato

Agora, já com as dimensões carregadas no Data Warehouse, podemos transformar e carregar a tabela `fato_metricas_crpt`. Para isso, seguimos os seguintes passos:

1. Lemos as surrogate keys das dimensões do Data Warehouse para DataFrames.
2. Fazemos *merge* entre o DataFrame da Fato e os DataFrames das dimensões para obter as SKs.
3. Renomeamos as colunas da Silver para padronizar com o que foi definido na camada Gold. Além disso tratamos valores `Infinity` e `NaN`, e selecionamos a ordem final das colunas.
4. Executamos `TRUNCATE` na Fato para garantir que quaiquer dados antigos sejam removidos. Em seguida, usamos `execute_values` para carregar o DataFrame direto na tabela `dw.fato_metricas_crpt`.

In [ ]:
print("Iniciando Transformação e Carga da Tabela Fato...")
try:
    with get_connection() as conn:
        df_data_lkp = pd.read_sql("SELECT sk_data, dt_cpta FROM dw.dim_data", conn)
        df_hora_lkp = pd.read_sql("SELECT sk_hora, hr_cpta FROM dw.dim_hora", conn)
        df_crpt_lkp = pd.read_sql("SELECT sk_crpt, nk_nome FROM dw.dim_crpt", conn)

    df_fato = df_silver.copy()
    df_fato['join_date'] = pd.to_datetime(df_fato['last_updated']).dt.date
    df_fato['join_time'] = pd.to_datetime(df_fato['last_updated']).dt.floor('S').dt.time
    df_data_lkp['dt_cpta'] = pd.to_datetime(df_data_lkp['dt_cpta']).dt.date
    df_hora_lkp['hr_cpta'] = pd.to_datetime(df_hora_lkp['hr_cpta'], format='%H:%M:%S').dt.time

    df_fato = df_fato.merge(df_crpt_lkp, left_on='name', right_on='nk_nome', how='left')\
                     .merge(df_data_lkp, left_on='join_date', right_on='dt_cpta', how='left')\
                     .merge(df_hora_lkp, left_on='join_time', right_on='hr_cpta', how='left')

    df_fato['sk_crpt'] = df_fato['sk_crpt'].fillna(unknowns['crpt']).astype(int)
    df_fato['sk_data'] = df_fato['sk_data'].fillna(unknowns['data']).astype(int)
    df_fato['sk_hora'] = df_fato['sk_hora'].fillna(unknowns['hora']).astype(int)

    rename_cols = {
        'cmc_rank': 'rnk_cmc', 'price': 'vlr_preco_usd', 'volume_24h': 'vlr_volume_24h',
        'market_cap': 'vlr_mktcap', 'dominance': 'vlr_dmn', 'market_pair_count': 'qtd_pairs',
        'circulating_supply': 'qtd_circ_sup', 'total_supply': 'qtd_tot_sup',
        'fully_dillutted_market_cap': 'vlr_fd_mktcap', 'market_cap_by_total_supply': 'vlr_mcap_ts',
        'ytd_price_change_percentage': 'pc_ytd', 'percent_change_1h': 'pc_1h', 'percent_change_24h': 'pc_24h',
        'percent_change_7d': 'pc_7d', 'percent_change_30d': 'pc_30d', 'percent_change_60d': 'pc_60d', 'percent_change_90d': 'pc_90d'
    }

    df_fato.rename(columns=rename_cols, inplace=True)
    df_fato['vlr_tovr'] = np.nan

    cols_final = [
        'sk_crpt', 'sk_data', 'sk_hora',
        'rnk_cmc', 'vlr_preco_usd', 'vlr_volume_24h', 'vlr_mktcap', 'vlr_dmn', 'vlr_tovr',
        'qtd_pairs', 'qtd_circ_sup', 'qtd_tot_sup', 'vlr_fd_mktcap', 'vlr_mcap_ts',
        'pc_ytd', 'pc_1h', 'pc_24h', 'pc_7d', 'pc_30d', 'pc_60d', 'pc_90d'
    ]

    df_fato_final = df_fato[cols_final].copy()

    df_fato_final.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_fato_list = df_fato_final.where(pd.notna(df_fato_final), None).values.tolist()

    print("Carregando dados na dw.fato_metricas_crpt...")
    with get_connection() as conn, conn.cursor() as cur:
        cur.execute("TRUNCATE TABLE dw.fato_metricas_crpt RESTART IDENTITY;")
        query = sql.SQL("INSERT INTO dw.fato_metricas_crpt ({}) VALUES %s").format(sql.SQL(', ').join(map(sql.Identifier, cols_final)))
        
        print(f"Inserindo {len(df_fato_list)} registros na Fato... (Pode demorar)")
        execute_values(cur, query, df_fato_list, page_size=1000)
        conn.commit()

    print(f"Fato carregada: {len(df_fato_list)} registros.")
except Exception as e:
    print(f"Erro ao carregar fato: {e}")
    raise

## Validação final

Esta seção executa uma contagem de linhas em todas as tabelas para verificar o resultado final. 

In [ ]:
print("Verificando contagem de linhas final...")
try:
    with get_connection() as conn:
        query = """
        SELECT 'dw.dim_crpt' AS Tabela, COUNT(*) AS Total_Linhas FROM dw.dim_crpt
        UNION ALL
        SELECT 'dw.dim_data' AS Tabela, COUNT(*) AS Total_Linhas FROM dw.dim_data
        UNION ALL
        SELECT 'dw.dim_hora' AS Tabela, COUNT(*) AS Total_Linhas FROM dw.dim_hora
        UNION ALL
        SELECT 'dw.fato_metricas_crpt' AS Tabela, COUNT(*) AS Total_Linhas FROM dw.fato_metricas_crpt;
        """
        df_contagem = pd.read_sql_query(query, conn)
        
        print("\n--- CONTAGEM FINAL DE LINHAS ---")
        display(df_contagem)
except Exception as e:
    print(f"Erro ao verificar contagens: {e}")

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import requests
import json
import time
import os

df = pd.read_csv("../data_layer/silver/silver_currencies_data.csv")
df.to_sql(
    "fato_metricas_crpt",
    conn,
    schema="dw",
    if_exists="append",
    index=False
)


TENANT = "SEU_TENANT_ID"
CLIENT = "SEU_CLIENT_ID"
SECRET = "SEU_SECRET"
GROUP = "SEU_WORKSPACE_ID"

token = requests.post(
    f"https://login.microsoftonline.com/{TENANT}/oauth2/v2.0/token",
    data={
        "client_id": CLIENT,
        "client_secret": SECRET,
        "scope": "https://analysis.windows.net/powerbi/api/.default",
        "grant_type": "client_credentials"
    }
).json()["access_token"]

headers = {
    "Authorization": f"Bearer {token}"
}

# =========================================================
# 4) Publicar PBIX no Power BI Online (Linux)
# =========================================================

PBIX = "crypto_model.pbix"  # seu template pbix

with open(PBIX, "rb") as f:
    resp = requests.post(
        f"https://api.powerbi.com/v1.0/myorg/groups/{GROUP}/imports?datasetDisplayName=DW_Crypto",
        headers=headers,
        files={"file": f}
    )

import_id = resp.json()["id"]

# =========================================================
# 5) Obter ID do Dataset importado
# =========================================================

time.sleep(5)

imports = requests.get(
    f"https://api.powerbi.com/v1.0/myorg/groups/{GROUP}/imports",
    headers=headers
).json()

DATASET_ID = imports["value"][0]["datasets"][0]["id"]

# =========================================================
# 6) Atualizar o dataset publicado
# =========================================================

requests.post(
    f"https://api.powerbi.com/v1.0/myorg/groups/{GROUP}/datasets/{DATASET_ID}/refreshes",
    headers=headers
)
